In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

pd.set_option('display.max_columns', None)
plt.rc('font', family='NanumBarunGothic')

train = pd.read_csv('train.csv')
submission = pd.read_csv('sample_submission.csv')
holiday = pd.read_csv('holidays_2023_2025.csv')

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

## **ishHoliday, is_before_holiday, is_after_holiday, is_weekend 파생변수 생성**

In [2]:
train['영업일자'] = pd.to_datetime(train['영업일자'])
train['년'] = train['영업일자'].dt.year.astype(int)
train['월'] = train['영업일자'].dt.month.astype(int)
train['일'] = train['영업일자'].dt.day.astype(int)
train['요일'] = train['영업일자'].dt.weekday.astype(int)

In [3]:
train.head()

,영업일자,영업장명_메뉴명,매출수량,년,월,일,요일
0,2023-01-01,느티나무 셀프BBQ_1인 수저세트,0,2023,1,1,6
1,2023-01-02,느티나무 셀프BBQ_1인 수저세트,0,2023,1,2,0
2,2023-01-03,느티나무 셀프BBQ_1인 수저세트,0,2023,1,3,1
3,2023-01-04,느티나무 셀프BBQ_1인 수저세트,0,2023,1,4,2
4,2023-01-05,느티나무 셀프BBQ_1인 수저세트,0,2023,1,5,3


In [4]:
holiday['locdate'] = pd.to_datetime(holiday['locdate'])

In [5]:
holiday.head()

,name,locdate,isHoliday
0,1월1일,2023-01-01,Y
1,설날,2023-01-21,Y
2,설날,2023-01-22,Y
3,설날,2023-01-23,Y
4,대체공휴일,2023-01-24,Y


In [6]:
holiday = holiday.rename(columns={'isHoliday': 'is_holiday'})
train = pd.merge(train, holiday[['locdate', 'is_holiday']], how='left', left_on='영업일자', right_on='locdate')
train['is_holiday'] = train['is_holiday'].fillna('N')
train['is_holiday'] = train['is_holiday'].apply(lambda x: 1 if x == 'Y' else 0)
display(train.head())

,영업일자,영업장명_메뉴명,매출수량,년,월,일,요일,locdate,is_holiday
0,2023-01-01,느티나무 셀프BBQ_1인 수저세트,0,2023,1,1,6,2023-01-01,1
1,2023-01-02,느티나무 셀프BBQ_1인 수저세트,0,2023,1,2,0,NaT,0
2,2023-01-03,느티나무 셀프BBQ_1인 수저세트,0,2023,1,3,1,NaT,0
3,2023-01-04,느티나무 셀프BBQ_1인 수저세트,0,2023,1,4,2,NaT,0
4,2023-01-05,느티나무 셀프BBQ_1인 수저세트,0,2023,1,5,3,NaT,0


In [7]:
train = train.drop(['locdate'], axis=1)
display(train.head())

,영업일자,영업장명_메뉴명,매출수량,년,월,일,요일,is_holiday
0,2023-01-01,느티나무 셀프BBQ_1인 수저세트,0,2023,1,1,6,1
1,2023-01-02,느티나무 셀프BBQ_1인 수저세트,0,2023,1,2,0,0
2,2023-01-03,느티나무 셀프BBQ_1인 수저세트,0,2023,1,3,1,0
3,2023-01-04,느티나무 셀프BBQ_1인 수저세트,0,2023,1,4,2,0
4,2023-01-05,느티나무 셀프BBQ_1인 수저세트,0,2023,1,5,3,0


In [8]:
train['is_before_holiday'] = train['is_holiday'].shift(1).fillna(0)
train['is_before_holiday'] = train['is_before_holiday'].astype(int)
train['is_after_holiday'] = train['is_holiday'].shift(-1).fillna(0)
train['is_after_holiday'] = train['is_after_holiday'].astype(int)
train['is_weekend'] = train['요일'].apply(lambda x: 1 if x in [5, 6] else 0)
display(train.head())

,영업일자,영업장명_메뉴명,매출수량,년,월,일,요일,is_holiday,is_before_holiday,is_after_holiday,is_weekend
0,2023-01-01,느티나무 셀프BBQ_1인 수저세트,0,2023,1,1,6,1,0,0,1
1,2023-01-02,느티나무 셀프BBQ_1인 수저세트,0,2023,1,2,0,0,1,0,0
2,2023-01-03,느티나무 셀프BBQ_1인 수저세트,0,2023,1,3,1,0,0,0,0
3,2023-01-04,느티나무 셀프BBQ_1인 수저세트,0,2023,1,4,2,0,0,0,0
4,2023-01-05,느티나무 셀프BBQ_1인 수저세트,0,2023,1,5,3,0,0,0,0


## **banquet_type, meal_only 파생변수 생성**

In [9]:
train[['영업장명','메뉴명']] = train['영업장명_메뉴명'].str.split('_', n=1, expand = True)
train = train.drop(['영업장명_메뉴명'], axis=1)
display(train.head())

,영업일자,매출수량,년,월,일,요일,is_holiday,is_before_holiday,is_after_holiday,is_weekend,영업장명,메뉴명
0,2023-01-01,0,2023,1,1,6,1,0,0,1,느티나무 셀프BBQ,1인 수저세트
1,2023-01-02,0,2023,1,2,0,0,1,0,0,느티나무 셀프BBQ,1인 수저세트
2,2023-01-03,0,2023,1,3,1,0,0,0,0,느티나무 셀프BBQ,1인 수저세트
3,2023-01-04,0,2023,1,4,2,0,0,0,0,느티나무 셀프BBQ,1인 수저세트
4,2023-01-05,0,2023,1,5,3,0,0,0,0,느티나무 셀프BBQ,1인 수저세트


In [10]:
df_연회장 = train[train['영업장명']=='연회장'].pivot_table(index='영업일자',columns='메뉴명',values='매출수량', aggfunc = 'sum').reset_index()
df_연회장.head()


메뉴명,영업일자,Cass Beer,Conference L1,Conference L2,Conference L3,Conference M1,Conference M8,Conference M9,Convention Hall,Cookie Platter,Grand Ballroom,OPUS 2,Regular Coffee,골뱅이무침,공깃밥,돈목살 김치찌개 (밥포함),로제 치즈떡볶이,마라샹궈,매콤 무뼈닭발&계란찜,모둠 돈육구이(3인),삼겹살추가 (200g),야채추가,왕갈비치킨,주먹밥 (2ea)
0,2023-01-01,0,0,0,0,0,0,0,0,0,0,0,0,9,0,3,6,0,0,11,0,1,0,4
1,2023-01-02,0,0,0,0,0,0,0,0,0,0,0,0,9,0,2,1,0,2,6,0,1,0,1
2,2023-01-03,0,1,0,0,0,0,0,1,0,0,0,0,4,0,0,1,0,4,6,0,0,0,2
3,2023-01-04,0,0,0,0,0,0,0,1,0,0,0,0,4,0,7,3,0,1,12,0,1,0,3
4,2023-01-05,0,0,0,1,0,0,0,1,0,0,1,0,7,0,4,4,0,1,7,0,1,0,0


In [13]:
df_연회장['연회장 대여'] = df_연회장[['Conference L1','Conference L2','Conference L3','Conference M1','Conference M8','Conference M9','Convention Hall','Grand Ballroom','OPUS 2']].sum(axis=1)
df_연회장['음료 및 쿠키'] = df_연회장[['Cookie Platter','Cass Beer','Regular Coffee']].sum(axis=1)
df_연회장['음식'] = df_연회장[['골뱅이무침','공깃밥','돈목살 김치찌개 (밥포함)','로제 치즈떡볶이','마라샹궈','매콤 무뼈닭발&계란찜','모둠 돈육구이(3인)','삼겹살추가 (200g)','야채추가','왕갈비치킨','주먹밥 (2ea)']].sum(axis=1)

In [14]:
def banquet_type(row):
    if row['연회장 대여'] > 0 and row['음식'] == 0 and row['음료 및 쿠키'] == 0:
        return 1 # 대여만
    elif row['연회장 대여'] == 0 and row['음식'] == 0 and row['음료 및 쿠키'] > 0:
        return 2 # 음료및쿠키만
    elif row['연회장 대여'] == 0 and row['음식'] > 0 and row['음료 및 쿠키'] == 0:
        return 3 # 음식만
    elif row['연회장 대여'] > 0 and row['음식'] == 0 and row['음료 및 쿠키'] > 0:
        return 4 # 대여+음료및쿠키
    elif row['연회장 대여'] > 0 and row['음식'] > 0 and row['음료 및 쿠키'] == 0:
        return 5 # 대여+음식
    elif row['연회장 대여'] == 0 and row['음식'] > 0 and row['음료 및 쿠키'] > 0:
        return 6 # 음식+음료및쿠키
    elif row['연회장 대여'] > 0 and row['음식'] > 0 and row['음료 및 쿠키'] > 0:
        return 7 # 대여+음료및쿠키+음식
    else:
        return 0 # 연회장 총매출이 0인경우
df_연회장['banquet_type'] = df_연회장.apply(banquet_type, axis=1)

display(df_연회장.head())

메뉴명,영업일자,Cass Beer,Conference L1,Conference L2,Conference L3,Conference M1,Conference M8,Conference M9,Convention Hall,Cookie Platter,Grand Ballroom,OPUS 2,Regular Coffee,골뱅이무침,공깃밥,돈목살 김치찌개 (밥포함),로제 치즈떡볶이,마라샹궈,매콤 무뼈닭발&계란찜,모둠 돈육구이(3인),삼겹살추가 (200g),야채추가,왕갈비치킨,주먹밥 (2ea),연회장 대여,음료 및 쿠키,음식,banquet_type
0,2023-01-01,0,0,0,0,0,0,0,0,0,0,0,0,9,0,3,6,0,0,11,0,1,0,4,0,0,34,3
1,2023-01-02,0,0,0,0,0,0,0,0,0,0,0,0,9,0,2,1,0,2,6,0,1,0,1,0,0,22,3
2,2023-01-03,0,1,0,0,0,0,0,1,0,0,0,0,4,0,0,1,0,4,6,0,0,0,2,2,0,17,5
3,2023-01-04,0,0,0,0,0,0,0,1,0,0,0,0,4,0,7,3,0,1,12,0,1,0,3,1,0,31,5
4,2023-01-05,0,0,0,1,0,0,0,1,0,0,1,0,7,0,4,4,0,1,7,0,1,0,0,3,0,24,5


In [15]:
display(df_연회장['banquet_type'].value_counts())

,count
banquet_type,
4,160
7,116
3,102
5,74
0,53
1,23
6,2
2,2


4(대여 + 음료 및 쿠키)가 160개로 가장 많은 조합.
7(대여 + 음식 + 음료 및 쿠키)가 116개로 두 번째.
3(음식만)이 102개로 세 번째.
5(대여 + 음식)이 74개로 네 번째.
0(총매출없음)이 53개로 다섯 번째.
1(대여만)이 23개로 여섯 번째.
6(음식+음료 및 쿠키)가 2개, 2(음료및 쿠키만)이 2개로 하위권.

=> **대여와 함께 음식/음료및쿠키를 각각 곁들이거나 음식만 하는 경우, 총매출없음가 있는 듯**